In [0]:
%python 

#### Use this code to configure your accoutn before running SQL in Databricks

storage_account_name = 'rgdevglobalreahl'
storage_account_key =  'RjnFSNg2IDzQ5bN/8aTZOFK1lnpL6zytdMnuQIrJQMw6psa7++18fCoIfaLkq5DEFV3Hp0b7WZjR+AStQuhR+A=='

spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", storage_account_key)


from pyspark.sql.functions import expr, array, lit, explode, col, concat, coalesce, when, to_date, date_format
import pandas as pd


from pyspark.sql.functions import col, lit, round 
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window
import logging
from functools import reduce


In [0]:
%sql
DROP VIEW IF EXISTS MS_Claims;

CREATE TEMPORARY VIEW MS_Claims 
USING PARQUET
OPTIONS (
  
  path = "wasbs://marketscan@rgdevglobalreahl.blob.core.windows.net/marketscan/commercial/CCAE_A"
  );


select *
from MS_Claims
-- where MS_Source_File != 'Inpatient'
-- and bill_type is null
limit 10
  
  
-- IP Svc: 111,460,288
-- OP Svc: 751,900,874



In [0]:
##### Cobine Tables and write to parquet
########Use same code below for commerical and medicare. Just switch the folder when running each

op_services_path = "wasbs://marketscan@rgdevglobalreahl.blob.core.windows.net/marketscan/commercial/CCAE_O"
op_services = spark.read.parquet(op_services_path, header=True)


ip_services_path = "wasbs://marketscan@rgdevglobalreahl.blob.core.windows.net/marketscan/commercial/CCAE_S"
ip_services = spark.read.parquet(ip_services_path, header=True)

facility_header_path = "wasbs://marketscan@rgdevglobalreahl.blob.core.windows.net/marketscan/commercial/CCAE_F"
facility_header = spark.read.parquet(facility_header_path, header=True)  


### Add source file
op_services = op_services.withColumn("MS_Source_File", lit("Outpatient"))
ip_services = ip_services.withColumn("MS_Source_File", lit("Inpatient"))

### union
combined_services = reduce(lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True), [op_services, ip_services])


###join facility header
joined_df = combined_services.join(facility_header.select('FACHDID', 'ENROLID', 'MSCLMID', 'DX5', 'DX6', 'DX7', 'DX8', 'DX9', 'BILLTYP', 'PROC2', 'PROC3', 'PROC4', 'PROC5', 'PROC6'), on=['FACHDID', 'ENROLID', 'MSCLMID'], how='left')

##### Reorder and Rename

##### Rename columns
column_mappings = {'FACHDID' : 'fachdid' , 
'ENROLID' : 'member_id' , 
'MSCLMID' : 'claim_number' , 
'SEQNUM' : 'seqnum' , 
'VERSION' : 'marketscan_version_number' , 
'DX1' : 'icd_diagnosis_code1' , 
'DX2' : 'icd_diagnosis_code2' , 
'PROC1' : 'procedure_code' , 
'PROCTYP' : 'proctyp' , 
'EFAMID' : 'family_id' , 
'REVCODE' : 'revenue_code' , 
'SVCDATE' : 'incurred_date' , 
'DOBYR' : 'dob_year' , 
'AGE' : 'member_age' , 
'CAP_SVC' : 'cap_svc' , 
'COB' : 'cob' , 
'COINS' : 'coins' , 
'COPAY' : 'copay' , 
'DEDUCT' : 'deduct' , 
'DX3' : 'icd_diagnosis_code3' , 
'DX4' : 'icd_diagnosis_code4' , 
'DXVER' : 'dxver' , 
'FACPROF' : 'facprof' , 
'MHSACOVG' : 'mhsacovg' , 
'NETPAY' : 'paid' , 
'NTWKPROV' : 'ntwkprov' , 
'PAIDNTWK' : 'paid_inn' , 
'PAY' : 'allowed' , 
'PDDATE' : 'paid_date' , 
'PLANTYP' : 'plan_type' , 
'PROCGRP' : 'proc_group' , 
'PROCMOD' : 'cpt_modifier' , 
'PROVID' : 'provider_id' , 
'QTY' : 'quantity' , 
'SVCSCAT' : 'service_category' , 
'TSVCDAT' : 'service_to_date' , 
'MDC' : 'MDC' , 
'REGION' : 'employee_region' , 
'MSA' : 'employee_msa' , 
'STDPLAC' : 'pos' , 
'STDPROV' : 'provider_type' , 
'DATATYP' : 'data_type' , 
'AGEGRP' : 'age_group' , 
'EECLASS' : 'employee_class' , 
'EESTATU' : 'employee_status' , 
'EGEOLOC' : 'employee_geo' , 
'EIDFLAG' : 'eidflag' , 
'EMPREL' : 'employee_relation' , 
'ENRFLAG' : 'enrflag' , 
'PHYFLAG' : 'physician_specialty' , 
'RX' : 'rx' , 
'SEX' : 'member_gender' , 
'HLTHPLAN' : 'healthplan_vs_employer' , 
'INDSTRY' : 'industry' , 
'NPI' : 'npi' , 
'UNITS' : 'units' , 
'MEDADV' : 'medicare_advantage_flag' , 
'MSN_VERSION' : 'msn_version' , 
'CASEID' : 'caseid' , 
'DISDATE' : 'discharge_date' , 
'ADMDATE' : 'admit_date' , 
'DRG' : 'DRG' , 
'PDX' : 'principal_diagnosis' , 
'PPROC' : 'principal_procedure' , 
'ADMTYP' : 'admit_type' , 
'DSTATUS' : 'discharge_status' , 
'DX5' : 'icd_diagnosis_code5' , 
'DX6' : 'icd_diagnosis_code6' , 
'DX7' : 'icd_diagnosis_code7' , 
'DX8' : 'icd_diagnosis_code8' , 
'DX9' : 'icd_diagnosis_code9' , 
'BILLTYP' : 'bill_type' , 
'PROC2' : 'procedure_code2' , 
'PROC3' : 'procedure_code3' , 
'PROC4' : 'procedure_code4' , 
'PROC5' : 'procedure_code5' , 
'PROC6' : 'procedure_code6' , 
'YEAR' : 'year' , 
'MS_Source_File' : 'MS_Source_File' }

#### Rename columns
for old_name, new_name in column_mappings.items():
    joined_df = joined_df.withColumnRenamed(old_name, new_name)


# Reorder columns

new_order = ['fachdid',
'member_id',
'claim_number',
'seqnum',
'marketscan_version_number',
'family_id',
'principal_diagnosis',
'icd_diagnosis_code1',
'icd_diagnosis_code2',
'icd_diagnosis_code3',
'icd_diagnosis_code4',
'icd_diagnosis_code5',
'icd_diagnosis_code6',
'icd_diagnosis_code7',
'icd_diagnosis_code8',
'icd_diagnosis_code9',
'principal_procedure',
'procedure_code',
'procedure_code2',
'procedure_code3',
'procedure_code4',
'procedure_code5',
'procedure_code6',
'cpt_modifier',
'bill_type',
'admit_type',
'discharge_status',
'revenue_code',
'DRG',
'cob',
'coins',
'copay',
'deduct',
'paid',
'allowed',
'quantity',
'units',
'paid_inn',
'provider_id',
'npi',
'service_to_date',
'paid_date',
'incurred_date',
'discharge_date',
'admit_date',
'dob_year',
'member_age',
'cap_svc',
'proctyp',
'dxver',
'facprof',
'mhsacovg',
'ntwkprov',
'plan_type',
'proc_group',
'service_category',
'MDC',
'employee_region',
'employee_msa',
'pos',
'provider_type',
'data_type',
'age_group',
'employee_class',
'employee_status',
'employee_geo',
'eidflag',
'employee_relation',
'enrflag',
'physician_specialty',
'rx',
'member_gender',
'healthplan_vs_employer',
'industry',
'medicare_advantage_flag',
'msn_version',
'caseid',
'year',
'MS_Source_File' ]

joined_df = joined_df.select(*new_order)



##### Write to parquet
target_path = "wasbs://marketscan@rgdevglobalreahl.blob.core.windows.net/marketscan/commercial/marketscan-preprod-step2"
joined_df.write.partitionBy("YEAR", "MS_Source_File").parquet(target_path, mode="overwrite")


RX Table


In [0]:
Rx_path = "wasbs://marketscan@rgdevglobalreahl.blob.core.windows.net/marketscan/medicare/MDCR_D"
Rx_services = spark.read.parquet(Rx_path, header=True)


##### Rename columns
column_mappings = {'SEQNUM' : 'seqnum' , 
'VERSION' : 'marketscan_version_number' , 
'EFAMID' : 'family_id' , 
'ENROLID' : 'member_id' , 
'NDCNUM' : 'ndc' , 
'SVCDATE' : 'incurred_date' , 
'DOBYR' : 'dob_year' , 
'YEAR' : 'year' , 
'AGE' : 'member_age' , 
'AWP' : 'awp' , 
'CAP_SVC' : 'cap_svc' , 
'COB' : 'cob' , 
'COINS' : 'coins' , 
'COPAY' : 'copay' , 
'DAYSUPP' : 'days_supply' , 
'DEDUCT' : 'deduct' , 
'DISPFEE' : 'disp_fee' , 
'GENERID' : 'generic_product_id' , 
'INGCOST' : 'ingredient_cost' , 
'METQTY' : 'metric_quantity' , 
'MHSACOVG' : 'mhsacovg' , 
'NETPAY' : 'paid' , 
'NTWKPROV' : 'ntwkprov' , 
'PAIDNTWK' : 'paid_inn' , 
'PAY' : 'allowed' , 
'PDDATE' : 'paid_date' , 
'PHARMID' : 'pharmacy_id' , 
'PLANTYP' : 'plan_type' , 
'QTY' : 'quantity' , 
'REFILL' : 'refill_number' , 
'RXMR' : 'mail_order_ind' , 
'SALETAX' : 'sales_tax' , 
'THERCLS' : 'ther_class' , 
'DAWIND' : 'daw_ind' , 
'DEACLAS' : 'dea_class' , 
'GENIND' : 'generic_equivalent' , 
'MAINTIN' : 'maintenance_ind' , 
'THERGRP' : 'therapeutic_group' , 
'REGION' : 'employee_region' , 
'MSA' : 'employee_msa' , 
'DATATYP' : 'data_type' , 
'AGEGRP' : 'age_group' , 
'EECLASS' : 'employee_class' , 
'EESTATU' : 'employee_status' , 
'EGEOLOC' : 'employee_geo' , 
'EIDFLAG' : 'eidflag' , 
'EMPREL' : 'employee_relation' , 
'ENRFLAG' : 'enrflag' , 
'PHYFLAG' : 'physician_specialty' , 
'SEX' : 'member_gender' , 
'HLTHPLAN' : 'healthplan_vs_employer' , 
'INDSTRY' : 'industry' , 
'MEDADV' : 'medicare_advantage_flag' , 
'MSN_VERSION' : 'msn_version'  }

#### Rename columns
for old_name, new_name in column_mappings.items():
    Rx_services = Rx_services.withColumnRenamed(old_name, new_name)

##### Write to parquet
target_path = "wasbs://marketscan@rgdevglobalreahl.blob.core.windows.net/marketscan/medicare/marketscan-Rx-preprod-step2"

Rx_services.write.partitionBy("year").parquet(target_path, mode="overwrite")

Medicaid


In [0]:
%sql
DROP VIEW IF EXISTS MS_Claims;

CREATE TEMPORARY VIEW MS_Claims 
USING PARQUET
OPTIONS (
  
  path = "wasbs://marketscan@rgdevglobalreahl.blob.core.windows.net/marketscan/commercial/CCAE_F"
  );


select *
from MS_Claims
-- where MS_Source_File != 'Inpatient'
-- and bill_type is null
limit 10
  

SEQNUM,VERSION,DX1,DX2,PROC1,FACHDID,EFAMID,ENROLID,DOBYR,YEAR,AGE,BILLTYP,CAP_SVC,CASEID,COB,COINS,COPAY,DEDUCT,DX3,DX4,DX5,DX6,DX7,DX8,DX9,DXVER,MHSACOVG,NETPAY,NTWKPROV,PAIDNTWK,PDDATE,PLANTYP,PROC2,PROC3,PROC4,PROC5,PROC6,PROVID,SVCDATE,TSVCDAT,MDC,DSTATUS,REGION,MSA,STDPLAC,STDPROV,DATATYP,AGEGRP,EECLASS,EESTATU,EGEOLOC,EIDFLAG,EMPREL,ENRFLAG,PHYFLAG,RX,SEX,HLTHPLAN,INDSTRY,MSCLMID,NPI,POADX1,POADX2,POADX3,POADX4,POADX5,POADX6,POADX7,POADX8,POADX9,MEDADV,MSN_VERSION
56843216,10,O24424,O34211,10D00Z1,61724298,41696048,4169604801,1981,2022,41,111,N,571342,0.0,0.0,0.0,0.0,O9089,Z370,Z3A37,null,null,null,null,0,null,49.0,Y,Y,2022-08-05,6,3E0T3BZ,3E0T3BZ,null,null,null,null,2022-06-29,2022-07-01,14,01,1,35084,21,320,1,3,1,1,11,1,1,1,1,1,2,0,4,2503997,null,Y,Y,null,1,1,null,null,null,null,0,221
22999367,10,K8062,E6601,null,24526334,47685970,4768597001,1985,2022,37,137,N,null,0.0,0.0,0.0,0.0,Z20822,Z6842,null,null,null,null,null,0,1,10.53,null,Y,2023-02-10,6,null,null,null,null,null,null,2022-11-20,2022-11-20,07,01,2,null,22,1,1,3,5,1,15,1,1,1,1,1,2,0,2,517263,ybPUohqytG,null,null,null,null,null,null,null,null,null,0,221
56843217,10,O24424,O34211,10D00Z1,61724299,41696048,4169604801,1981,2022,41,111,N,571342,0.0,0.0,0.0,0.0,O9089,Z370,Z3A37,null,null,null,null,0,null,135.0,Y,Y,2022-08-05,6,3E0T3BZ,3E0T3BZ,null,null,null,null,2022-06-29,2022-07-01,14,01,1,35084,21,320,1,3,1,1,11,1,1,1,1,1,2,0,4,2503997,null,Y,Y,null,1,1,null,null,null,null,0,221
22999357,10,K8062,E6601,null,24526324,47685970,4768597001,1985,2022,37,137,N,null,0.0,0.0,0.0,0.0,Z20822,Z6842,null,null,null,null,null,0,1,0.0,null,Y,2023-02-10,6,null,null,null,null,null,null,2022-11-20,2022-11-20,07,01,2,null,22,1,1,3,5,1,15,1,1,1,1,1,2,0,2,517263,ybPUohqytG,null,null,null,null,null,null,null,null,null,0,221
56843218,10,O24424,O34211,10D00Z1,61724300,41696048,4169604801,1981,2022,41,111,N,571342,0.0,0.0,0.0,0.0,O9089,Z370,Z3A37,null,null,null,null,0,null,78.0,Y,Y,2022-08-05,6,3E0T3BZ,3E0T3BZ,null,null,null,null,2022-06-29,2022-07-01,14,01,1,35084,21,320,1,3,1,1,11,1,1,1,1,1,2,0,4,2503997,null,Y,Y,null,1,1,null,null,null,null,0,221
22999358,10,K8062,E6601,null,24526325,47685970,4768597001,1985,2022,37,137,N,null,0.0,0.0,0.0,0.0,Z20822,Z6842,null,null,null,null,null,0,1,0.0,null,Y,2023-02-10,6,null,null,null,null,null,null,2022-11-20,2022-11-20,07,01,2,null,22,1,1,3,5,1,15,1,1,1,1,1,2,0,2,517263,ybPUohqytG,null,null,null,null,null,null,null,null,null,0,221
22999359,10,K8062,E6601,null,24526326,47685970,4768597001,1985,2022,37,137,N,null,0.0,0.0,0.0,0.0,Z20822,Z6842,null,null,null,null,null,0,1,0.0,null,Y,2023-02-10,6,null,null,null,null,null,null,2022-11-20,2022-11-20,07,01,2,null,22,1,1,3,5,1,15,1,1,1,1,1,2,0,2,517263,ybPUohqytG,null,null,null,null,null,null,null,null,null,0,221
22999360,10,K8062,E6601,null,24526327,47685970,4768597001,1985,2022,37,137,N,null,0.0,0.0,0.0,0.0,Z20822,Z6842,null,null,null,null,null,0,1,67.0,null,Y,2023-02-10,6,null,null,null,null,null,null,2022-11-20,2022-11-20,07,01,2,null,22,1,1,3,5,1,15,1,1,1,1,1,2,0,2,517263,ybPUohqytG,null,null,null,null,null,null,null,null,null,0,221
22999361,10,K8062,E6601,null,24526328,47685970,4768597001,1985,2022,37,137,N,null,0.0,0.0,0.0,0.0,Z20822,Z6842,null,null,null,null,null,0,1,3.22,null,Y,2023-02-10,6,null,null,null,null,null,null,2022-11-20,2022-11-20,07,01,2,null,22,1,1,3,5,1,15,1,1,1,1,1,2,0,2,517263,ybPUohqytG,null,null,null,null,null,null,null,null,null,0,221
56843233,10,R0789,R002,null,61724315,41696060,4169606002,1981,2022,40,131,N,null,0.0,0.0,0.0,0.0,R030,null,null,null,null,null,null,0,null,46.64,Y,Y,2022-02-10,6,null,null,null,null,null,null,2022-01-28,2022-01-28,05,01,3,12580,22,220,1,3,5,1,35,1,2,1,1,1,2,0,4,2504009,null,1,1,1,null,null,null,null,null,null,0,221


Medicaid Med Claims

In [0]:
##### Cobine Tables and write to parquet


op_services_path = "wasbs://marketscan@rgdevglobalreahl.blob.core.windows.net/marketscan/medicaid/MDCD_O"
op_services = spark.read.parquet(op_services_path, header=True)


ip_services_path = "wasbs://marketscan@rgdevglobalreahl.blob.core.windows.net/marketscan/medicaid/MDCD_S"
ip_services = spark.read.parquet(ip_services_path, header=True)

facility_header_path = "wasbs://marketscan@rgdevglobalreahl.blob.core.windows.net/marketscan/medicaid/MDCD_F"
facility_header = spark.read.parquet(facility_header_path, header=True)  

ltc_services_path = "wasbs://marketscan@rgdevglobalreahl.blob.core.windows.net/marketscan/medicaid/MDCD_L"
ltc_services = spark.read.parquet(ltc_services_path, header=True)



### Add source file
op_services = op_services.withColumn("MS_Source_File", lit("Outpatient"))
ip_services = ip_services.withColumn("MS_Source_File", lit("Inpatient"))
ltc_services = ltc_services.withColumn("MS_Source_File", lit("LTC"))

### union
combined_services = reduce(lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True), [op_services, ip_services, ltc_services])


###join facility header
joined_df = combined_services.join(facility_header.select('FACHDID', 'ENROLID', 'MSCLMID', 'DX5', 'DX6', 'DX7', 'DX8', 'DX9', 'BILLTYP', 'PROC2', 'PROC3', 'PROC4', 'PROC5', 'PROC6'), on=['FACHDID', 'ENROLID', 'MSCLMID'], how='left')

##### Reorder and Rename

##### Rename columns

column_mappings = {'FACHDID' : 'fachdid' , 
'ENROLID' : 'member_id' , 
'MSCLMID' : 'claim_number' , 
'SEQNUM' : 'seqnum' , 
'MCASENUM' : 'mcasenum' , 
'DOBYR' : 'dob_year' , 
'DRUGCOVG' : 'rx' , 
'MHSACOVG' : 'mhsacovg' , 
'PLANTYP' : 'plan_type' , 
'BOE' : 'basis_of_eligibility' , 
'CAP' : 'capitation' , 
'MAS' : 'maintenance_assistance_status' , 
'MEDICARE' : 'medicare_dual_flag' , 
'VERSION' : 'marketscan_version_number' , 
'COB' : 'cob' , 
'COINS' : 'coins' , 
'COPAY' : 'copay' , 
'DEDUCT' : 'deduct' , 
'NETPAY' : 'paid' , 
'PAY' : 'allowed' , 
'FACPROF' : 'facprof' , 
'REVCODE' : 'revenue_code' , 
'DX1' : 'icd_diagnosis_code1' , 
'DX2' : 'icd_diagnosis_code2' , 
'DXVER' : 'dxver' , 
'PROC1' : 'procedure_code' , 
'PROCTYP' : 'proctyp' , 
'SEX' : 'member_gender' , 
'SVCDATE' : 'incurred_date' , 
'DX3' : 'icd_diagnosis_code3' , 
'DX4' : 'icd_diagnosis_code4' , 
'PDDATE' : 'paid_date' , 
'PROCGRP' : 'proc_group' , 
'QTY' : 'quantity' , 
'SVCSCAT' : 'service_category' , 
'TSVCDAT' : 'service_to_date' , 
'STDRACE' : 'stdrace' , 
'MDC' : 'MDC' , 
'STDPLAC' : 'pos' , 
'STDPROV' : 'provider_type' , 
'DENTAL' : 'dental_claim_indicator' , 
'PROV_ID' : 'prov_id' , 
'NPI' : 'npi' , 
'UNITS' : 'units' , 
'PROCMOD' : 'cpt_modifier' , 
'MSN_VERSION' : 'msn_version' , 
'DISDATE' : 'discharge_date' , 
'CASEID' : 'caseid' , 
'DRG' : 'DRG' , 
'PDX' : 'principal_diagnosis' , 
'PPROC' : 'principal_procedure' , 
'ADMTYP' : 'admit_type' , 
'ADMDATE' : 'admit_date' , 
'DSTATUS' : 'discharge_status' , 
'DX5' : 'icd_diagnosis_code5' , 
'DX6' : 'icd_diagnosis_code6' , 
'DX7' : 'icd_diagnosis_code7' , 
'DX8' : 'icd_diagnosis_code8' , 
'DX9' : 'icd_diagnosis_code9' , 
'BILLTYP' : 'bill_type' , 
'PROC2' : 'procedure_code2' , 
'PROC3' : 'procedure_code3' , 
'PROC4' : 'procedure_code4' , 
'PROC5' : 'procedure_code5' , 
'PROC6' : 'procedure_code6' , 
'YEAR' : 'year' , 
'MS_Source_File' : 'MS_Source_File' }

#### Rename columns
for old_name, new_name in column_mappings.items():
    joined_df = joined_df.withColumnRenamed(old_name, new_name)


# Reorder columns

new_order = ['fachdid',
'member_id',
'claim_number',
'seqnum',
'marketscan_version_number',
'principal_diagnosis',
'icd_diagnosis_code1',
'icd_diagnosis_code2',
'icd_diagnosis_code3',
'icd_diagnosis_code4',
'icd_diagnosis_code5',
'icd_diagnosis_code6',
'icd_diagnosis_code7',
'icd_diagnosis_code8',
'icd_diagnosis_code9',
'principal_procedure',
'procedure_code',
'procedure_code2',
'procedure_code3',
'procedure_code4',
'procedure_code5',
'procedure_code6',
'cpt_modifier',
'bill_type',
'admit_type',
'discharge_status',
'revenue_code',
'DRG',
'cob',
'coins',
'copay',
'deduct',
'paid',
'allowed',
'quantity',
'units',
'npi',
'service_to_date',
'paid_date',
'incurred_date',
'discharge_date',
'admit_date',
'dob_year',
'proctyp',
'dxver',
'facprof',
'mhsacovg',
'plan_type',
'proc_group',
'service_category',
'MDC',
'pos',
'provider_type',
'member_gender',
'msn_version',
'caseid',
'mcasenum',
'rx',
'basis_of_eligibility',
'capitation',
'maintenance_assistance_status',
'medicare_dual_flag',
'stdrace',
'dental_claim_indicator',
'prov_id',
'year',
'MS_Source_File']

joined_df = joined_df.select(*new_order)


##### Write to parquet
target_path = "wasbs://marketscan@rgdevglobalreahl.blob.core.windows.net/marketscan/medicaid/marketscan-preprod-step2"
joined_df.write.partitionBy("YEAR", "MS_Source_File").parquet(target_path, mode="overwrite")


In [0]:
%sql
DROP VIEW IF EXISTS MS_Claims;

CREATE TEMPORARY VIEW MS_Claims 
USING PARQUET
OPTIONS (
  path = "wasbs://marketscan@rgdevglobalreahl.blob.core.windows.net/marketscan/medicaid/MDCD_D"
  );


select *
from MS_Claims
limit 10

-- LTC 9,174,361
-- OP 736,270,920
-- IP 98,122,511
-- Total: 843,597,792

SEQNUM,ENROLID,MCASENUM,DOBYR,DRUGCOVG,MHSACOVG,PLANTYP,BOE,CAP,MAS,MEDICARE,VERSION,YEAR,AWP,COB,COINS,COPAY,DEDUCT,DAWIND,DISPFEE,INGCOST,QTY,REFILL,SVCDATE,DAYSUPP,GENERID,METQTY,NDCNUM,NETPAY,PAY,PDDATE,THERCLS,STDRACE,DEACLAS,MAINTIN,SEX,THERGRP,GENIND,PHARM_ID,MSN_VERSION
49805329,20051198599,null,1983,1,0,2,9,0,9,0,10,2023,1702.85,0.0,0.0,0.0,0.0,01,10.25,1702.85,1,0,2023-04-18,30,131888,30.0,59148003813,1372.61,1372.61,2023-04-24,70,1,6,1,2,08,1,iW2yo5tT1,202310
49805330,20051198599,null,1983,1,0,2,9,0,9,0,10,2023,129.79,0.0,0.0,0.0,0.0,01,10.25,129.8,1,0,2023-04-18,30,119279,30.0,13668013601,11.65,11.65,2023-04-24,69,1,6,2,2,08,4,iW2yo5tT1,202310
49805331,20051198599,null,1983,1,0,2,9,0,9,0,10,2023,436.2,0.0,0.0,0.0,0.0,01,10.25,436.2,1,1,2023-04-25,60,114353,120.0,42806008660,25.92,25.92,2023-05-01,75,1,6,3,2,08,4,iW2yo5tT1,202310
17955817,20035883115,null,1972,1,1,2,2,0,1,0,10,2023,2196.6,0.0,0.0,0.0,0.0,null,12.69,0.0,1,1,2023-08-03,30,129262,60.0,00832058111,47.1,34.41,2023-08-11,64,1,4,2,1,08,4,oLteTFsaeATM,202310
49805332,20051198599,null,1983,1,0,2,9,0,9,0,10,2023,138.65,0.0,0.0,0.0,0.0,01,10.25,138.65,1,0,2023-04-27,30,100069,120.0,65862067805,12.65,12.65,2023-05-01,74,1,4,3,2,08,4,iW2yo5tT1,202310
17955802,20035883115,null,1972,1,1,2,2,0,1,0,10,2023,59.5,0.0,0.0,0.0,0.0,null,12.49,0.0,1,1,2023-06-04,30,103857,45.0,50111056102,15.35,2.86,2023-06-09,69,1,6,2,1,08,4,oLteTFsaeATM,202310
49805333,20051198599,null,1983,1,0,2,9,0,9,0,10,2023,534.28,0.0,0.0,0.0,0.0,10,10.25,543.44,1,0,2023-05-02,30,115990,30.0,50458058701,437.3,437.3,2023-05-08,71,1,2,2,2,08,3,iW2yo5tT1,202310
17955803,20035883115,null,1972,1,1,2,2,0,1,0,10,2023,1068.17,0.0,0.0,0.0,0.0,null,12.49,0.0,1,1,2023-06-04,30,127005,60.0,62332017360,35.73,23.24,2023-06-09,68,1,5,1,1,08,4,oLteTFsaeATM,202310
49805334,20051198599,null,1983,1,0,2,9,0,9,0,10,2023,885.43,0.0,0.0,0.0,0.0,01,9.31,885.43,1,0,2023-05-08,28,135125,1.0,55513084301,718.99,718.99,2023-05-15,77,1,6,1,2,08,1,Qt2ya5tTQ,202310
17955804,20035883115,null,1972,1,1,2,2,0,1,0,10,2023,2196.6,0.0,0.0,0.0,0.0,null,12.49,0.0,1,1,2023-06-04,30,129262,60.0,00832058111,56.17,43.68,2023-06-09,64,1,4,2,1,08,4,oLteTFsaeATM,202310


Medicaid Rx Claims

In [0]:
Rx_path = "wasbs://marketscan@rgdevglobalreahl.blob.core.windows.net/marketscan/medicaid/MDCD_D"
Rx_services = spark.read.parquet(Rx_path, header=True)


##### Rename columns
column_mappings = {'SEQNUM' : 'seqnum' , 
'ENROLID' : 'member_id' , 
'MCASENUM' : 'mcasenum' , 
'DOBYR' : 'dob_year' , 
'DRUGCOVG' : 'rx' , 
'MHSACOVG' : 'mhsacovg' , 
'PLANTYP' : 'plan_type' , 
'BOE' : 'basis_of_eligibility' , 
'CAP' : 'capitation' , 
'MAS' : 'maintenance_assistance_status' , 
'MEDICARE' : 'medicare_dual_flag' , 
'VERSION' : 'marketscan_version_number' , 
'YEAR' : 'year' , 
'AWP' : 'awp' , 
'COB' : 'cob' , 
'COINS' : 'coins' , 
'COPAY' : 'copay' , 
'DEDUCT' : 'deduct' , 
'DAWIND' : 'daw_ind' , 
'DISPFEE' : 'disp_fee' , 
'INGCOST' : 'ingredient_cost' , 
'QTY' : 'quantity' , 
'REFILL' : 'refill_number' , 
'SVCDATE' : 'incurred_date' , 
'DAYSUPP' : 'days_supply' , 
'GENERID' : 'generic_product_id' , 
'METQTY' : 'metric_quantity' , 
'NDCNUM' : 'ndc' , 
'NETPAY' : 'paid' , 
'PAY' : 'allowed' , 
'PDDATE' : 'paid_date' , 
'THERCLS' : 'ther_class' , 
'STDRACE' : 'stdrace' , 
'DEACLAS' : 'dea_class' , 
'MAINTIN' : 'maintenance_ind' , 
'SEX' : 'member_gender' , 
'THERGRP' : 'therapeutic_group' , 
'GENIND' : 'generic_equivalent' , 
'PHARM_ID' : 'pharmacy_id' , 
'MSN_VERSION' : 'msn_version'  }

#### Rename columns
for old_name, new_name in column_mappings.items():
    Rx_services = Rx_services.withColumnRenamed(old_name, new_name)

##### Write to parquet
target_path = "wasbs://marketscan@rgdevglobalreahl.blob.core.windows.net/marketscan/medicaid/marketscan-Rx-preprod-step2"
Rx_services.write.partitionBy("year").parquet(target_path, mode="overwrite")



In [0]:
%sql
DROP VIEW IF EXISTS MS_Claims;

CREATE TEMPORARY VIEW MS_Claims 
USING PARQUET
OPTIONS (
  path = "wasbs://marketscan@rgdevglobalreahl.blob.core.windows.net/marketscan/medicaid/marketscan-Rx-preprod-step2"
  );


select *
from MS_Claims
limit 10


seqnum,member_id,mcasenum,dob_year,rx,mhsacovg,plan_type,basis_of_eligibility,capitation,maintenance_assistance_status,medicare_dual_flag,marketscan_version_number,awp,cob,coins,copay,deduct,daw_ind,disp_fee,ingredient_cost,quantity,refill_number,incurred_date,days_supply,generic_product_id,metric_quantity,ndc,paid,allowed,paid_date,ther_class,stdrace,dea_class,maintenance_ind,member_gender,therapeutic_group,generic_equivalent,pharmacy_id,msn_version,year
98564763,20051458435,null,1939,1,0,2,9,0,9,0,10,125.68,0.0,0.0,0.0,0.0,01,13.4,150.81,1,1,2022-12-29,30,112106,30.0,16714071302,14.8,14.8,2023-01-02,234,1,6,1,1,29,4,h12yG52Tl,202210,2022
98564764,20051458435,null,1939,1,0,2,9,0,9,0,10,397.96,0.0,0.0,0.0,0.0,02,13.4,477.55,1,1,2022-12-29,30,126336,60.0,00456141030,331.58,331.58,2023-01-09,51,1,6,1,1,07,3,h12yG52Tl,202210,2022
98564765,20051458435,null,1939,1,0,2,9,0,9,0,10,42.27,0.0,0.0,0.0,0.0,01,13.4,50.73,1,1,2022-12-29,30,108169,60.0,69367018005,14.38,14.38,2023-01-02,174,1,6,1,1,20,4,h12yG52Tl,202210,2022
98564766,20051458435,null,1939,1,0,2,9,0,9,0,10,121.29,0.0,0.0,0.0,0.0,01,13.4,145.55,1,1,2022-12-29,30,111573,30.0,75834025501,14.36,14.36,2023-01-02,53,1,6,1,1,07,4,h12yG52Tl,202210,2022
98564737,20051458435,null,1939,1,0,2,9,0,9,0,10,75.27,0.0,0.0,0.0,0.0,01,13.4,90.32,1,1,2022-06-07,30,108167,16.0,00054327099,16.46,16.46,2022-06-13,138,1,6,2,1,16,4,h12yG52Tl,202210,2022
93910054,20055566487,zV2WQlfsoi5Hd,1961,1,1,2,2,0,9,0,10,173.21,0.0,0.0,0.0,0.0,01,10.63,2.34,1,1,2022-04-29,30,114748,30.0,70377003012,12.97,12.97,2022-05-11,53,1,6,1,1,07,4,FTbWTlysGvtip,202210,2022
93910035,20055566476,jP2WQl0s8h2xd,1957,1,1,2,2,0,9,0,10,28.86,0.0,0.0,0.0,0.0,01,10.63,3.17,1,0,2022-08-28,3,100447,18.0,10702025350,13.8,13.8,2022-09-07,62,2,6,3,1,08,4,UTbWTlesov2hp,202210,2022
93910036,20055566476,jP2WQl0s8h2xd,1957,1,1,2,2,0,9,0,10,49.22,0.0,0.0,0.0,0.0,01,10.63,0.69,1,0,2022-08-29,30,100963,30.0,29300041519,11.32,11.32,2022-09-07,29,2,6,3,1,04,4,ETbWTlysot0h3,202210,2022
93910058,20055566487,zV2WQlfsoi5Hd,1961,1,1,2,2,0,9,0,10,46.7,0.0,0.0,0.0,0.0,01,10.63,2.03,1,0,2022-10-31,5,124911,6.0,68180016011,12.66,12.66,2022-11-09,9,1,6,3,1,02,4,FTbWTlysGvtip,202210,2022
93910038,20055566476,jP2WQl0s8h2xd,1957,1,1,2,2,0,9,0,10,9.02,0.0,0.0,0.0,0.0,01,10.63,1.43,1,0,2022-08-29,30,102261,30.0,45802065078,9.69,9.69,2022-09-07,1,2,7,2,1,01,6,ETbWTlysot0h3,202210,2022


Supplemental Files


In [0]:
%sql
DROP VIEW IF EXISTS MS_Claims;

CREATE TEMPORARY VIEW MS_Claims 
USING CSV
OPTIONS (
  path = "wasbs://marketscan-supplemental@rgdevglobalreahl.blob.core.windows.net/file1_med_only_2019_2023.csv",
  -- path = "wasbs://marketscan-supplemental@rgdevglobalreahl.blob.core.windows.net/file2_med_pharm_2019_2023.csv",
  header = true
  );


select *
from MS_Claims
limit 10


com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: shaded.databricks.org.apache.hadoop.fs.azure.AzureException: shaded.databricks.org.apache.hadoop.fs.azure.AzureException: Unable to access container marketscan-supplemental in account rgdevglobalreahl.blob.core.windows.net using anonymous credentials, and no credentials found for them  in the configuration.
	at shaded.databricks.org.apache.hadoop.fs.azure.AzureNativeFileSystemStore.createAzureStorageSession(AzureNativeFileSystemStore.java:1219)
	at shaded.databricks.org.apache.hadoop.fs.azure.AzureNativeFileSystemStore.initialize(AzureNativeFileSystemStore.java:560)
	at shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem.initialize(NativeAzureFileSystem.java:1397)
	at com.databricks.common.filesystem.LokiFileSystem$.getLokiFS(LokiFileSystem.scala:182)
	at com.databricks.common.filesystem.LokiFileSystem.initialize(LokiFileSystem.scala:253)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(Fi